<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Databehandling-Pradeep/Bioco_Databehandling_av_manglende_verdier_og_utstikkere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lasting av nødvendige pakker og bibliotek

In [ ]:
! pip install pyod

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.1.2-py3-none-any.whl size=190292 sha256=3b25192cc965da1d46d65e2437cda01f40b5839cc9db8ed97a7beea499800609
  Stored in directory: /root/.cache/pip/wheels/81/1b/61/aa85b78c3c0c8871f4231e3f4a03bb23cecb7db829498380ee
Successfully built pyod


Importering av nødvendige pakker og bibliotek

In [ ]:
import pandas as pd
import numpy as np

from pyod.models.hbos import HBOS

Importering av data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Velger første kolonne med dato og tid som index
bioco_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/førbehandlet_bioco_data.csv',
                         header=0,
                         sep=',',
                         index_col=0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Oppdeling av data til prediktorer- og responsvariabler

In [ ]:
# Datasett for prediktorer
prediktor_data = bioco_data.iloc[:, :-4]

# Verdier for prediktorer
X = prediktor_data.values

# Datasett for hver kvalitetsmåling
collagen = bioco_data['Collagen']
mw = bioco_data['Mw']
smallmolecules = bioco_data['SmallMolecules']
brixadjusted = bioco_data['BrixAdjusted']

# Behandling av manglende verdier

Oversikt over antall manglende verider for per kolonne

In [36]:
bioco_data.isnull().sum()

EnzymeType_A1         0
EnzymeType_A2         0
EnzymeType_B          0
EnzymeType_C          0
EnzymeType_D          0
EnzymeType_E          0
RawMatFlow            0
WaterFlow             0
RawMatPercent         0
NIRfat            14132
NIRprotein        15676
NIRash            13921
NIRwater          13903
TT07                  0
TT08                  0
PT03                  0
TT20                  0
TT09                  0
TT12                  0
Collagen          43097
Mw                42656
SmallMolecules    42656
BrixAdjusted      42656
dtype: int64

Oversikt over rader med manglende verdier for RawMatflow

In [ ]:
# Henter ut index til rader med manglende verdier for "RawMatFlow"
rawmat_manglende_index = bioco_data[bioco_data['RawMatFlow'].isnull()].index

# Sjekker hvilke andre kolonner som har manglende verdier for samme rader
bioco_data[rawmat_manglende_index[0]:rawmat_manglende_index[-1]].isnull().sum()

# Sletter de spesifikke rader fra datasett
bioco_data.drop(bioco_data.loc[rawmat_manglende_index[0]:rawmat_manglende_index[-1]].index,
                inplace=True)

Oversikt over rader med manglende verdier for NIRfat

In [42]:
# Henter ut index til rader med manglende verdier for "NIRfat"
nirfat_manglende_index = bioco_data[bioco_data['NIRfat'].isnull()].index

# Inspeksjon av mulige ekstremverdier

## Ekstremverdi inspeksjon med HBOS

In [ ]:
hbos = HBOS()
hbos.fit(X)

y_predikert = hbos.labels_
y_scores = hbos.decision_scores_

ValueError: Input contains NaN.

Oppdeling av original datasett for hver kvalitetsmåling

In [ ]:
# Trekker ut prediktorer som et eget datasett
prediktor_data = bioco_rå_data.iloc[:, :-4]

# Datasett for "Collagen"
collagen_datasett = pd.concat([prediktor,
                               bioco_rå_data['Collagen']],
                              axis=1)

# Datasett for "Mw"
mw_datasett = pd.concat([prediktor,
                         bioco_rå_data['Mw']],
                        axis=1)

# Datasett for "SmallMolecules"
smallmolecules_datasett = pd.concat([prediktor,
                                     bioco_rå_data['SmallMolecules']],
                                    axis=1)

# Datasett for "BrixAdjusted"
brixadjusted_datasett = pd.concat([prediktor,
                                   bioco_rå_data['BrixAdjusted']],
                                  axis=1)

Lagrer hvert datasett som csv

In [ ]:
# Collagen datasett som csv fil
collagen_datasett.to_csv('')